In [ ]:
%reload_ext autoreload
%autoreload 2

# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.plotting.backend = 'holoviews'
from tqdm import tqdm

import sklearn
print(sklearn.__version__)
from sklearn.utils import parallel_backend
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.model_selection import GridSearchCV, LeavePOut, LeaveOneOut, cross_validate, KFold

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
)
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import TweedieRegressor

from sklearn.metrics import r2_score

from sklearn import set_config
# set_config(transform_output='pandas')  # only works for sklearn >= 1.2

import prepare_data
from components import PCOA
#from helpers import PipelineHelper, SMWrapper
from settings import Config, shortnames, target
from plots import scatter_chart
from cv import generate_feature_sets

## Data preparation

In [ ]:
%%capture
# cell magic to supress output. Comment it out to see output of this cell.

# What happened so far: DB extract and blank procedure. Now import resulting MP data from csv
mp_pdd = prepare_data.get_pdd()

# Also import sediment data (sediment frequencies per size bin from master sizer export)
grainsize_iow, grainsize_cau = prepare_data.get_grainsizes()[0:2]
scor_iow = PCOA(grainsize_iow, 2)[0]
scor_cau = PCOA(grainsize_cau, 2)[0]

# ...some data wrangling to prepare particle domain data and sample domain data for MP and combine with certain sediment aggregates.
sdd_iow = prepare_data.aggregate_SDD(mp_pdd)
sdd_iow = prepare_data.additional_sdd_merging(sdd_iow, how='outer')
sdd_iow = sdd_iow.merge(scor_iow, right_index=True, left_on='Sample', how='outer')
sdd_iow = sdd_iow.replace({'Sample': shortnames}).sort_values(by='Sample')

sdd_cau = pd.read_csv('../data/Metadata_CAU_sampling_log.csv', index_col=0).join(pd.read_csv('../data/GRADISTAT_CAU_vol_log-cau_closed.csv', index_col=0), how='outer')
sdd_cau = sdd_cau.merge(scor_cau, right_index=True, left_on='Sample', how='outer').reset_index()

In [ ]:
## Additional variable generation (e.g. predictor derivatives)
 
# sdd_iow['Dist_WWTP_revsq'] = ((1/sdd_iow['Dist_WWTP'])**3)*10000000000  # calculates the squared of the reversed Distance 
# sdd_iow['Dist_WWTP_revsq'] = (((sdd_iow['Dist_WWTP'].max()-sdd_iow['Dist_WWTP'])+1)**3)/100000000000  # calculates the squared of the reversed Distance 
# sdd_iow['Dist_WWTP_revsq'] = ((sdd_iow['Dist_WWTP'].max()/sdd_iow['Dist_WWTP'])**3)/100  # calculates the squared of the reversed Distance
# sdd_iow['Dist_WWTP_revsq'] = ((sdd_iow['Dist_WWTP'].max()/sdd_iow['Dist_WWTP'])**2)  # calculates the squared of the reversed Distance

# sdd_iow

In [ ]:
## Split data into samples used for building the model and samples used for predicting.

model_data = sdd_iow.loc[~sdd_iow.Concentration.isna()].set_index('Sample')
pred_data = sdd_iow.loc[sdd_iow.Concentration.isna()]
pred_data = pd.concat([pred_data, sdd_cau.drop('Date',axis=1)]).set_index('Sample')

In [ ]:
## Potential outlier exclusion
# model_data = model_data.drop(['S10d', 'S02d', 'S04d', 'S01d'])

In [ ]:
## List of features (predictors) to be used in the model. Beware: depending on the preprocessing steps not all features might be used.

featurelist = [
    # 'Depth',
    # 'LON', 'LAT',
    # 'Dist_Land',
    # 'Dist_Marina',
    'Dist_WWTP',
    # 'WWTP_influence_as_tracer_mean_dist',
    # 'WWTP_influence_as_cumulated_residence',
    # 'WWTP_influence_as_mean_time_travelled',
    'WWTP_influence_as_tracer_mean_dist__sed_18µm_allseasons_444',
    'WWTP_influence_as_endpoints_mean_dist__sed_18µm_allseasons_444',
    'WWTP_influence_as_cumulated_residence__sed_18µm_allseasons_444',
    'WWTP_influence_as_mean_time_travelled__sed_18µm_allseasons_444',
    'WWTP_influence_as_tracer_mean_dist__nosed_18µm_spring_444',
    'WWTP_influence_as_endpoints_mean_dist__nosed_18µm_spring_444',
    'WWTP_influence_as_cumulated_residence__nosed_18µm_spring_444',
    'WWTP_influence_as_mean_time_travelled__nosed_18µm_spring_444',
    'WWTP_influence_as_tracer_mean_dist__nosed_allsizes_spring_444',
    'WWTP_influence_as_endpoints_mean_dist__nosed_allsizes_spring_444',
    'WWTP_influence_as_cumulated_residence__nosed_allsizes_spring_444',
    'WWTP_influence_as_mean_time_travelled__nosed_allsizes_spring_444',
    'WWTP_influence_as_tracer_mean_dist__sed_18µm_autumn_222',
    'WWTP_influence_as_endpoints_mean_dist__sed_18µm_autumn_222',
    'WWTP_influence_as_cumulated_residence__sed_18µm_autumn_222',
    'WWTP_influence_as_mean_time_travelled__sed_18µm_autumn_222',
    'WWTP_influence_as_tracer_mean_dist__sed_18µm_autumn_444',
    'WWTP_influence_as_endpoints_mean_dist__sed_18µm_autumn_444',
    'WWTP_influence_as_cumulated_residence__sed_18µm_autumn_444',
    'WWTP_influence_as_mean_time_travelled__sed_18µm_autumn_444',
    'WWTP_influence_as_tracer_mean_dist__nosed_18µm_autumn_222',
    'WWTP_influence_as_endpoints_mean_dist__nosed_18µm_autumn_222',
    'WWTP_influence_as_cumulated_residence__nosed_18µm_autumn_222',
    'WWTP_influence_as_mean_time_travelled__nosed_18µm_autumn_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_allsizes_autumn_222',
    'WWTP_influence_as_endpoints_mean_dist__nosed_allsizes_autumn_222',
    'WWTP_influence_as_cumulated_residence__nosed_allsizes_autumn_222',
    'WWTP_influence_as_mean_time_travelled__nosed_allsizes_autumn_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_allsizes_allseasons_444',
    'WWTP_influence_as_endpoints_mean_dist__nosed_allsizes_allseasons_444',
    'WWTP_influence_as_cumulated_residence__nosed_allsizes_allseasons_444',
    'WWTP_influence_as_mean_time_travelled__nosed_allsizes_allseasons_444',
    'WWTP_influence_as_tracer_mean_dist__sed_allsizes_allseasons_444',
    'WWTP_influence_as_endpoints_mean_dist__sed_allsizes_allseasons_444',
    'WWTP_influence_as_cumulated_residence__sed_allsizes_allseasons_444',
    'WWTP_influence_as_mean_time_travelled__sed_allsizes_allseasons_444',
    'WWTP_influence_as_tracer_mean_dist__sed_18µm_allseasons_222',
    'WWTP_influence_as_endpoints_mean_dist__sed_18µm_allseasons_222',
    'WWTP_influence_as_cumulated_residence__sed_18µm_allseasons_222',
    'WWTP_influence_as_mean_time_travelled__sed_18µm_allseasons_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_18µm_allseasons_222',
    'WWTP_influence_as_endpoints_mean_dist__nosed_18µm_allseasons_222',
    'WWTP_influence_as_cumulated_residence__nosed_18µm_allseasons_222',
    'WWTP_influence_as_mean_time_travelled__nosed_18µm_allseasons_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_18µm_spring_222',
    'WWTP_influence_as_endpoints_mean_dist__nosed_18µm_spring_222',
    'WWTP_influence_as_cumulated_residence__nosed_18µm_spring_222',
    'WWTP_influence_as_mean_time_travelled__nosed_18µm_spring_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_allsizes_allseasons_222',
    'WWTP_influence_as_endpoints_mean_dist__nosed_allsizes_allseasons_222',
    'WWTP_influence_as_cumulated_residence__nosed_allsizes_allseasons_222',
    'WWTP_influence_as_mean_time_travelled__nosed_allsizes_allseasons_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_allsizes_autumn_444',
    'WWTP_influence_as_endpoints_mean_dist__nosed_allsizes_autumn_444',
    'WWTP_influence_as_cumulated_residence__nosed_allsizes_autumn_444',
    'WWTP_influence_as_mean_time_travelled__nosed_allsizes_autumn_444',
    'WWTP_influence_as_tracer_mean_dist__sed_18µm_spring_444',
    'WWTP_influence_as_endpoints_mean_dist__sed_18µm_spring_444',
    'WWTP_influence_as_cumulated_residence__sed_18µm_spring_444',
    'WWTP_influence_as_mean_time_travelled__sed_18µm_spring_444',
    'WWTP_influence_as_tracer_mean_dist__nosed_18µm_autumn_444',
    'WWTP_influence_as_endpoints_mean_dist__nosed_18µm_autumn_444',
    'WWTP_influence_as_cumulated_residence__nosed_18µm_autumn_444',
    'WWTP_influence_as_mean_time_travelled__nosed_18µm_autumn_444',
    'WWTP_influence_as_tracer_mean_dist__sed_18µm_spring_222',
    'WWTP_influence_as_endpoints_mean_dist__sed_18µm_spring_222',
    'WWTP_influence_as_cumulated_residence__sed_18µm_spring_222',
    'WWTP_influence_as_mean_time_travelled__sed_18µm_spring_222',
    'WWTP_influence_as_tracer_mean_dist__sed_allsizes_allseasons_222',
    'WWTP_influence_as_endpoints_mean_dist__sed_allsizes_allseasons_222',
    'WWTP_influence_as_cumulated_residence__sed_allsizes_allseasons_222',
    'WWTP_influence_as_mean_time_travelled__sed_allsizes_allseasons_222',
    'WWTP_influence_as_tracer_mean_dist__nosed_allsizes_spring_222',
    'WWTP_influence_as_endpoints_mean_dist__nosed_allsizes_spring_222',
    'WWTP_influence_as_cumulated_residence__nosed_allsizes_spring_222',
    'WWTP_influence_as_mean_time_travelled__nosed_allsizes_spring_222',
    # 'Dist_WWTP2',
    # 'Dist_WWTP_revsq',
    # 'MODE 1 (µm)',
    # 'D10 (µm)',
    # 'D50 (µm)',
    # 'D90 (µm)',
    # 'perc GRAVEL',
    # 'perc SAND',
    # 'perc MUD',
    # 'perc CLAY',
    # 'OM_D50',
    # 'TOC',
    # 'Hg',
    # 'TIC',
    # 'regio_sep',
    'PC1',
    # 'PC2'
    ]
model_X = model_data[featurelist]
model_y = model_data[target]
pred_X = pred_data[featurelist]

In [ ]:
# Scale data using StandardScaler

scaler = StandardScaler()
model_X.values[:] = scaler.fit_transform(model_X)
pred_X.values[:] = scaler.transform(pred_X)

In [ ]:
## Mutual exclusive list (list of lists detailing predictors that are not allowed together in one model candidate)

mutual_exclusive = [
    ['D50 (µm)', 'PC1'],
    ['D50 (µm)', 'perc MUD'],
    ['perc MUD', 'PC1'],
    ['perc MUD', 'TOC'],
]
exclusive_keywords = ['WWTP']  # only feature_candidates sets with max 1 feature containing each keyword will be considered

In [ ]:
## Check some basic statistics of the target variable

# model_y.describe()
# model_y.hist()
# model_X.info()


## Model building

### Custom preprocessing functions to be used in the model pipeline

In [ ]:
## Create exhaustive feature selector, using leave-p-out on columns labels to generate a boolean matrix.

# min_features = 2  # minimum number of features to be used in the model

# feature_candidates_list = []
# for i in tqdm(range(1,len(model_X.columns)+1-min_features)):
#     lpo = LeavePOut(p=i)
#     # base_index = [False] * len(model_X.columns)
#     for candidate_indices, _ in lpo.split(model_X.columns):
#         feature_candidates = model_X.columns[candidate_indices]
#         if any(all(pd.Series(ex_feats).isin(feature_candidates)) for ex_feats in mutual_exclusive):
#             continue  # if all entries of any row in mutual_exclusive list are present in the current feature_candidates, then don't put them in the feature_candidates_list
#         feature_candidates_list.append(feature_candidates)
# if not any(all(pd.Series(ex_feats).isin(model_X.columns)) for ex_feats in mutual_exclusive):
#     feature_candidates_list.append(model_X.columns)  # also append the set of all possible features

feature_candidates_list = generate_feature_sets(model_X, mutual_exclusive, exclusive_keywords, num_feat=2, n_jobs=-1)

def SelectFeatures(model_X, feature_set, feature_sets=feature_candidates_list):
    return model_X.loc[:, feature_sets[feature_set]]

CustomFeatureSelector = FunctionTransformer(SelectFeatures)#, feature_names_out='one-to-one')

### Creating the model pipeline

In [ ]:
## Pipeline and parameter grid for model selection, see here for inspiration: https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

PreProcessor = ColumnTransformer([
      ('selector', CustomFeatureSelector, model_X.columns),
      # ('imputer', SimpleImputer(), make_column_selector(dtype_include=np.number)),
      # ('scaler', StandardScaler(), make_column_selector(dtype_include=np.number)),
      # ('encoder', OneHotEncoder(), make_column_selector(dtype_include=object)),
      ])

pipe = Pipeline(steps=[
    ('preprocessor', PreProcessor),
    ('regressor', DummyRegressor())
    ])

preprocessor_params = [{
    # 'preprocessor__selector': [CustomFeatureSelector],
      'preprocessor__selector__kw_args': [{'feature_set':i} for i in range(len(feature_candidates_list))],
    
    # 'preprocessor__scaler': [StandardScaler()],#MaxAbsScaler(), MinMaxScaler(), RobustScaler(), QuantileTransformer(), Normalizer()],
    #    'preprocessor__scaler__with_mean': [True],
    #    'preprocessor__scaler__with_std': [True],
    }]

regressor_params = [
    # {
    # 'regressor': [DummyRegressor()],
    #     'regressor__strategy': ['median'],
    # },
    
    # {
    # 'regressor': [SVR()],
    #    'regressor__C': [0.1, 1.5],
    #    'regressor__kernel': ['linear', 'rbf', 'poly'],
    #    'regressor__degree': [2, 3, 4, 5],
    # },

    {
    'regressor': [TweedieRegressor(max_iter=100000)],
    #   'regressor__power': [2],
      # 'regressor__power': [0, 1, 1.25, 1.5, 1.6, 1.7, 1.8, 1.9, 1.95, 1.99, 2, 3],
      # 'regressor__alpha': [0, 1], 
      'regressor__link': ['log'],#, 'identity', 'auto'],
      # 'regressor__fit_intercept': [True, False],
    #   'regressor__warm_start': [True, False],
    #    'regressor__fit_params__sample_weights': [None, model_data.loc[model_X.index, 'Mass'].to_numpy()],  # FIXME: fit_params seem not to be accepted from gridsearch params, only as argument in fit method directly...
    }
  
    # {
    # 'regressor': [RadiusNeighborsRegressor()],
    #     'regressor__radius': [1000, 10000, 100000],
    #     'regressor__weights': ['uniform', 'distance'],
    #     'regressor__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    #     'regressor__leaf_size': [10, 20, 30, 40, 50],
    # },
    
    # {
    # 'regressor': [RandomForestRegressor()],
    #      'regressor__n_estimators': [10, 50], #[20, 50, 100, 150],
    #      'regressor__max_features': [None, 'sqrt', 'log2'],
    #      'regressor__min_samples_split': [2, 10],
    #      'regressor__min_samples_leaf': [1, 5],
    #      'regressor__bootstrap': [True],
    #      'regressor__max_depth': [None, 5, 50],
    #     #  'regressor__warm_start': [True, False],
    # },
    
    # {
    # 'regressor': [GradientBoostingRegressor()],
        # 'regressor__loss': ['ls', 'lad', 'huber', 'quantile'],
        # 'regressor__learning_rate': [0.1, 0.5],  
        # 'regressor__n_estimators': [100, 200],
        # 'regressor__subsample': [0.5, 1.0],
        # 'regressor__criterion': ['friedman_mse', 'mse', 'mae'],
        # 'regressor__min_samples_split': [2, 10],
        # 'regressor__min_samples_leaf': [1, 5],
        # 'regressor__max_depth': [3, 5],
        # 'regressor__min_weight_fraction_leaf': [0.0, 0.1],
        # 'regressor__max_features': [None, 'sqrt', 'log2'],
        # 'regressor__max_leaf_nodes': [None, 5, 10],
        # 'regressor__min_impurity_decrease': [0.0, 0.1],
        # 'regressor__min_impurity_split': [None, 0.1],
        # 'regressor__alpha': [0.9, 0.95, 0.99, 0.999],
        # 'regressor__tol': [0.0001, 0.001, 0.01],
        # 'regressor__validation_fraction': [0.1, 0.2],
        # 'regressor__n_iter_no_change': [None, 5, 10],
        # 'regressor__ccp_alpha': [0.0, 0.1],
        # 'regressor__warm_start': [True, False],
    # },
]

params = [{**dict_pre, **dict_reg} for dict_reg in regressor_params for dict_pre in preprocessor_params]
# params = regressor_params

### Training the model

In [ ]:
## The pipeline is run by searching the provided paramter space using scorings of a crossvalidation technique to find out how each model candidate performs.

scoring = {
    'R2': 'r2',
    'MAPE': 'neg_mean_absolute_percentage_error',
    'MedAE': 'neg_median_absolute_error',
    'MSLE': 'neg_mean_squared_log_error',
  }
refit_scorer = 'R2'

with parallel_backend('loky', n_jobs=-1):
    innerCV = GridSearchCV(
        pipe,
        params,
        scoring= scoring,
        refit= refit_scorer,
        cv=10,
        verbose=0,
        # n_jobs=-1
        )

    outerCV = cross_validate(
        innerCV,
        model_X,
        model_y,
        scoring=scoring,
        cv=10,
        # return_train_score=True,
        return_estimator=True,
        verbose=1,
        # n_jobs=-1
        )

## Evaluating the model

In [ ]:
# make a df of all outer cv results and show it sorted by the best scoring metric
outerCV_df = pd.DataFrame(outerCV)
outerCV_df.sort_values(by=f'test_{refit_scorer}', ascending=False, inplace=True)
outerCV_df.rename_axis(index='outerCV_fold', inplace=True)

## Get best model params for each of the outer cv folds:
best_params_df = pd.DataFrame()
for i, model in enumerate(outerCV['estimator']):
    best_params = model.best_params_
    # best_params_df = pd.concat([best_params_df, pd.DataFrame(best_params, index=[i])])  # this does not work when RandomForestRegressor is used, because some internals call len() on the values of the best_params dict, which raises AttributeError: 'RandomForestRegressor' object has no attribute 'estimators_'
    # instead filling df with for-loop...:
    current_best_params_df = pd.DataFrame()
    for key, value in best_params.items():
        current_best_params_df[key] = [value]
        current_best_params_df.index = [i]
    best_params_df = pd.concat([best_params_df, current_best_params_df])

results = outerCV_df.join(best_params_df)

In [ ]:
print('Number of coefficients in each outer folds best model: ', [outerCV['estimator'][i].best_estimator_.named_steps['regressor'].n_features_in_ for i in range(len(outerCV['estimator']))])
print('Coeffs: ', *[outerCV['estimator'][i].best_estimator_.named_steps['regressor'].coef_ for i in range(len(outerCV['estimator']))], sep='\n')
# results.estimator[0].best_estimator_.named_steps['preprocessor'].transformers[0][1].get_feature_names_out()

In [ ]:
outerCV['estimator'][0].best_estimator_.named_steps['regressor'].n_features_in_

In [ ]:
s = results.preprocessor__selector__kw_args.apply(lambda x: [x['feature_set'], feature_candidates_list[x['feature_set']]])
d = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T

In [ ]:
pd.set_option('display.max_colwidth', -1)
d

In [ ]:
results['estimator'].apply(lambda x: x.score(model_X, model_y))
# pd.DataFrame.from_dict(dict(zip(s.values)))


In [ ]:
r2_all_but_no_refit = results['estimator'].apply(lambda x: r2_score(model_y, x.predict(model_X)))
r2_all_but_no_refit

In [ ]:
results_refitted_to_all = results.copy(deep=True)
results_refitted_to_all['estimator'].apply(lambda x: r2_score(model_y, x.best_estimator_.fit(model_X, model_y).predict(model_X)))
r2_refitted_to_all = results_refitted_to_all['estimator'].apply(lambda x: r2_score(model_y, x.predict(model_X)))
r2_refitted_to_all

In [ ]:
## Printing score of the best performing model candidate and its parameters.

outer_fold = 9  # OBS: using only one of the outer-folds models here... e.g. [0]

print(f'{scoring[refit_scorer]}: {outerCV["estimator"][outer_fold].score(model_X, model_y)}')  
print(outerCV['estimator'][outer_fold].best_params_)

## Plotting the results of the best performing model candidate.
df = pd.DataFrame(zip(model_y, outerCV['estimator'][outer_fold].predict(model_X)), index=model_X.index, columns=[target, 'predicted'])
from plots import scatter_chart
scatter_chart(df.reset_index(), target, 'predicted',
                                 labels='Sample',
                                 identity=True,
                                 equal_axes=True,
                                #  xscale='log', yscale='log',
                                #  xtransform=True, ytransform=True,
                                 width=800, height=800,
                                 title='yhat vs. y')[0]


In [ ]:
# get inner cv results of the outer cv fold which achieved the best scoring metric
innerCV_df = pd.DataFrame(outerCV_df.loc[outerCV_df[f'test_score'].idxmax(), 'estimator'].cv_results_)
innerCV_df.sort_values(by=f'rank_test_{scoring[0]}', ascending=True)

In [ ]:
outerCV['estimator'][0].best_estimator_.named_steps['preprocessor'].get_feature_names_out()

In [ ]:
## Extracting feature names
# [grid.best_estimator_.named_steps['preprocessor'].named_transformers_['selector'].get_feature_names_out(input_features=model_X.columns.tolist())]

In [ ]:
## Show how the model performs on the training data

train_pred_y = grid.predict(model_X)  # use the best model to predict the data on the same samples that were used to train the model
print(f'R² = {r2_score(model_y, train_pred_y)}')  # adjusted R² = 1 - (1 - R²) * (n - 1) / (n - p) with n = number of samples, p = number of features

df = pd.concat([
    model_data.loc[model_y.index].regio_sep,
    model_y,
    pd.Series(
        train_pred_y,
        name='Prediction',
        index=model_y.index)
        ],
    axis=1
    ).reset_index()
    
scatter_chart(df, target, 'Prediction', color='regio_sep', labels='Sample', identity=True, equal_axes=False)[0]

In [ ]:
## Take a look at all model candidates and their performance

scores = pd.DataFrame(grid.cv_results_)
# scores